In [12]:
import requests
from time     import sleep
from selenium import webdriver
from bs4      import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from random import randrange

In [2]:
# https://itnext.io/overwrite-previously-printed-lines-4218a9563527
def print_refresh(string):
    print(string, 
          '\t\t\t\t\t\t\t\t\t\t\t\t',
          end='\r', 
          flush = True
         )

def pause_update(n, start='', end=''):
    for i in range(n):
        print_refresh(f'{start} Waiting {n} seconds: {n-i} seconds remaining{end}')
        sleep(1)
    print_refresh(f'{start}Finished with {n} second wait{end}')


In [20]:
''' Expand Deck Lists '''
def expand_deck_lists():
  button = driver.find_element( 
      By.CLASS_NAME, 'tournament-decklist-collapse'
  )
  button.click()
  print_refresh(f'expansion click finished')
  #wait.until(lambda _ : revealed.is_displayed() )
  #pause_update(120)

In [4]:
''' Find all cards '''
def grab_cards():
    temp = driver.find_elements( By.CLASS_NAME, 'card_name' )
    print( len(temp), 'cards found' )
    pause_update(3)
    return temp

## Preample Variables

In [5]:
card_dictionary = {}

In [7]:
events = [ # {'event page name' : number of pages}
            {'mythic-championship-vi'          : 5} ,
            {'mythic-championship-1-cleveland' : 2} ,
            {'pro-tour-guilds-of-ravnica'      : 2} ,
            {'pro-tour-dominaria'              : 2} ,
            {'pro-tour-ixalan'                 : 2} ,
            {'pro-tour-hour-of-devastation'    : 2} ,
            {'pro-tour-amonkhet'               : 2} ,
            {'pro-tour-aether-revolt'          : 2} ,
            {'pro-tour-kaladesh'               : 2} ,
            {'pro-tour-eldritch-moon'          : 1} ,
            {'pro-tour-shadows-over-innistrad' : 2} ,
            {'pro-tour-battle-for-zendikar'    : 1} ,
]

In [8]:
downloaded = []
with open('./logs/downloaded.list') as fi:
  for line in fi:
    downloaded.append( line.strip().strip("'") )

## Scrape

In [29]:
''' Initialize WebDriver '''
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.set_page_load_timeout(60)
driver.get('https://www.mtggoldfish.com/')
print('Driver Initiated')

Driver Initiated


In [16]:
new_events = events
new_events

[{'mythic-championship-vi': 5},
 {'mythic-championship-1-cleveland': 2},
 {'pro-tour-guilds-of-ravnica': 2},
 {'pro-tour-dominaria': 2},
 {'pro-tour-ixalan': 2},
 {'pro-tour-hour-of-devastation': 2},
 {'pro-tour-amonkhet': 2},
 {'pro-tour-aether-revolt': 2},
 {'pro-tour-kaladesh': 2},
 {'pro-tour-eldritch-moon': 1},
 {'pro-tour-shadows-over-innistrad': 2},
 {'pro-tour-battle-for-zendikar': 1}]

In [24]:
''' Begin Scraping '''
for dictionary in new_events:
  pause_update(30+randrange(15), start = f'preparing for next event')

  event_cards=[]
  for event, pages in dictionary.items():
    with open(f'./logs/{event}.cards', 'w') as fi:
      pass
    
    for page in range( pages ):
      tournament_URL = f'https://www.mtggoldfish.com/tournament/{event}?page={page+1}#paper'
      driver.get( tournament_URL )
      pause_update(30+randrange(15), start = f'checking {event} pg {page+1}')

      expand_deck_lists()
      pause_update(30+randrange(15), start = f'{event} pg {page+1} deck list expanded: ')

      cards = grab_cards()
      pause_update(15, start = f'{event} pg {page+1} cards grabbed' )

      for card in cards:
        if card mot in event_cards:
          event_cards.append( card )
          with open(f'./logs/{event}.cards', 'a') as fi:
            fi.write(f'{card.text}'\n)
            
        if card.text not in card_dictionary.keys():
          card_dictionary[card.text] = (
              card.find_element(By.TAG_NAME, 'a').get_attribute('href')
          )
          pause_update(3, start= f'adding {card.text}', end = f' dictionary updated: {len(card_dictionary)} unique cards')
  
      # check next page
      pause_update( 30+randrange(30), start = f'finished page {page+1} of {pages} for {event} ' )

    with open('./logs/card_grabbed_events.log', 'a') as log:
      log.write(f'{event}\n')

2614 cards foundhip-vi pg 1 deck list expanded: Finished with 33 second wait 																									
43 cards foundnship-vi pg 2 deck list expanded: Finished with 43 second wait 																									
2648 cards foundhip-vi pg 3 deck list expanded: Finished with 31 second wait 																									
43 cards foundnship-vi pg 4 deck list expanded: Finished with 39 second wait 																									
2532 cards foundhip-vi pg 5 deck list expanded: Finished with 42 second wait 																									
2526 cards foundhip-1-cleveland pg 1 deck list expanded: Finished with 39 second wait 																									
1017 cards foundhip-1-cleveland pg 2 deck list expanded: Finished with 34 second wait 																																					
2505 cards foundof-ravnica pg 1 deck list expanded: Finished with 43 second wait 																																				
1569 cards foundof-ravnica pg 2 deck list expanded: Finished with 43 second wait 																																									
2737 car

## Write to List for Event

In [ ]:
for cardname in card_dictionary.keys():
  with open(f'../{event}.cards', 'a') as fi:
    fi.write( f'{cardname}\n' )

## Check Against Already Downloaded Cards

In [25]:


print( len(downloaded) )

undownloaded_cards = []
for card in card_dictionary.keys():
  if card not in downloaded:
    undownloaded_cards.append( card )

print( f'{len( card_dictionary.keys() )} ?= {len( downloaded )} + {len( undownloaded_cards )}' )

988
1333 ?= 988 + 1285


## CSV Fetch for Event

In [26]:
just_downloaded = []

In [30]:
n = len(undownloaded_cards)

input('make sure you have logged in to MTG Goldfish and switched downloaded folders, then press enter')

for i, card_name in enumerate(undownloaded_cards):
  if card_name not in just_downloaded:
    status = f'fetching the #{i+1} of {n} card csv price data for {card_name}: '
    pause_update( 10 + randrange(5), start=status )
    driver.get( card_dictionary[card_name] )
    pause_update( 10 + randrange(5), start = f'loading site {card_name}, number {i+1} of {n}' )
    
    status = f'fetching {card_name}, {i+1} of {n} cards'
    pause_update( 10 + randrange(5), start = status )
    

    price_csv_url = driver.find_element(
      By.CLASS_NAME, 'price-history-download-container'
    ).find_element( By.TAG_NAME, 'a').get_attribute('href')
    driver.get( price_csv_url )

    status = f'data for {card_name} fetched, card {i+1} of {n} '
    just_downloaded.append(card_name)
    with open('./logs/downloaded.list', 'a') as log:
      log.write( f'{card_name}\n' )
    pause_update(10 + randrange(5), start = status)

print_refresh('finished')

make sure you have logged in to MTG Goldfish and switched downloaded folders, then press enter 
